# Setup & Configuration  


For setting up, the "requests" library is required. Additionally, in order to generate the returns in readable format, installing tabulate package is required to use the tabulat function. Moreover, to encode the location names, the function "quote" is required.

In [6]:
%pip install tabulate
import requests
from tabulate import tabulate
from urllib.parse import quote

# Weather Data Functions
The purpose of this function is to generate a response containing current weather conditions at a specific city, with a short forecast. The coding was prepared in a way that the get_weather(location, forecast_days) function calls the wttr.in service, parses the JSON response and gives the desired return.   


In [7]:
def _safe_int_conversion(value):
    """
    Safely converts a value to an integer, returning None if conversion fails.
    """
    try:
        return int(float(value)) # Convert to float first to handle decimal strings
    except (ValueError, TypeError):
        return None

def _format_current_weather_table(current_weather: dict) -> str:
    """
    Formats current weather data into a human-readable table.
    """
    headers = ["Metric", "Value"]
    table_data = [
        ["Temperature", f"{current_weather.get('temperature_celsius')}°C"],
        ["Feels Like", f"{current_weather.get('feels_like_celsius')}°C"],
        ["Description", current_weather.get('description')],
        ["Humidity", f"{current_weather.get('humidity_percent')}%"] if current_weather.get('humidity_percent') is not None else ["Humidity", "N/A"],
        ["Wind Speed", f"{current_weather.get('wind_speed_kmph')} km/h"] if current_weather.get('wind_speed_kmph') is not None else ["Wind Speed", "N/A"],
    ]
    return tabulate(table_data, headers=headers, tablefmt="grid")

def _format_forecast_table(forecast_data: list) -> str:
    """
    Formats forecast data into a human-readable table.
    """
    headers = ["Date", "Max Temp (°C)", "Min Temp (°C)", "Description"]
    table_data = []
    for day in forecast_data:
        table_data.append([
            day.get('date'),
            day.get('maxtemp_celsius'),
            day.get('mintemp_celsius'),
            day.get('description')
        ])
    return tabulate(table_data, headers=headers, tablefmt="grid")

def get_weather(location: str, forecast_days: int) -> dict:
    """
    Fetches current weather conditions and forecast data for a specified location
    using the wttr.in API.

    Args:
        location (str): The name of the city or location (e.g., "London", "New York").
        forecast_days (int): The number of forecast days to retrieve (1-5).

    Returns:
        dict: A dictionary containing current weather and forecast data formatted as tables,
              or an error message if the request fails or the location is invalid.
    """
    # 1. Input Validation
    if not 1 <= forecast_days <= 5:
        return {"error": "Forecast days must be between 1 and 5."}

    # 2. URL Encoding for Location
    # Encode the location to handle spaces and special characters in the URL.
    encoded_location = quote(location)

    # 3. Constructing the API URL
    # The format=j1 parameter requests JSON output.
    # The forecast parameter limits the number of forecast days.
    api_url = f"https://wttr.in/{encoded_location}?format=j1&forecast={forecast_days}"

    try:
        # 4. Making the API Request
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx).

        # 5. Parsing the JSON Response
        json_data = response.json()

        # 6. Error Handling for Invalid Location from API
        # wttr.in returns an "error" key if the location is not found.
        if "error" in json_data:
            return {"error": f"Location not found or invalid: {location}"}

        # 7. Extracting Current Weather Conditions
        current_condition_data = json_data.get("current_condition", [{}])[0]
        current_weather = {
            "temperature_celsius": _safe_int_conversion(current_condition_data.get("temp_C")),
            "description": current_condition_data.get("weatherDesc", [{}])[0].get("value") if current_condition_data.get("weatherDesc") else "N/A",
            "feels_like_celsius": _safe_int_conversion(current_condition_data.get("FeelsLikeC")),
            "humidity_percent": _safe_int_conversion(current_condition_data.get("humidity")),
            "wind_speed_kmph": _safe_int_conversion(current_condition_data.get("windspeedKmph")),
        }

        # 8. Extracting Forecast Data
        forecast_data_list = []
        for day_data in json_data.get("weather", [])[:forecast_days]:
            daily_forecast = {
                "date": day_data.get("date"),
                "maxtemp_celsius": _safe_int_conversion(day_data.get("maxtempC")),
                "mintemp_celsius": _safe_int_conversion(day_data.get("mintempC")),
                "description": day_data.get("hourly", [{}])[0].get("weatherDesc", [{}])[0].get("value") if day_data.get("hourly") else "N/A"
            }
            forecast_data_list.append(daily_forecast)

        # 9. Formatting data as tables
        current_weather_table = _format_current_weather_table(current_weather)
        forecast_table = _format_forecast_table(forecast_data_list)

        # 10. Returning the Processed Data
        return {
            "current_weather_table": current_weather_table,
            "forecast_table": forecast_table
        }

    except requests.exceptions.RequestException as e:
        # Handle network-related errors (e.g., connection refused, DNS error, HTTP errors).
        return {"error": f"Network error or API request failed: {e}"}
    except ValueError:
        # Handle JSON decoding errors if the response is not valid JSON.
        return {"error": "Failed to decode JSON response from API."}
    except Exception as e:
        # Catch any other unexpected errors for robustness.
        return {"error": f"An unexpected error occurred: {e}"}

The output generated is a formatted current weaher condition and forecast for selected number of days for the specific city.

In [12]:
weather_data = get_weather("Perth", 4)
print(weather_data['current_weather_table'])
print(weather_data['forecast_table'])

+-------------+---------------+
| Metric      | Value         |
+=============+===============+
| Temperature | 17°C          |
+-------------+---------------+
| Feels Like  | 17°C          |
+-------------+---------------+
| Description | Partly cloudy |
+-------------+---------------+
| Humidity    | 48%           |
+-------------+---------------+
| Wind Speed  | 10 km/h       |
+-------------+---------------+
+------------+-----------------+-----------------+--------------------+
| Date       |   Max Temp (°C) |   Min Temp (°C) | Description        |
+============+=================+=================+====================+
| 2025-09-16 |              16 |              11 | Patchy rain nearby |
+------------+-----------------+-----------------+--------------------+
| 2025-09-17 |              18 |              13 | Patchy rain nearby |
+------------+-----------------+-----------------+--------------------+
| 2025-09-18 |              16 |              14 | Patchy rain nearby |
+-------

#Visualization Functions

#Natural Language Processing

#User Interface

#Main Application Logic

#Testing and Examples